## Data Cleaning & Profiling

In [4]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/Users/ging/Documents/Data Analytics Course/GHG Emissions Worldwide/02 Data/Original Data/GHG_Emissions_World_Sector_Country.xlsx'
data = pd.read_excel(file_path)

# Initial profiling
missing_data = data.isnull().sum()
duplicates = data.duplicated().sum()

print("Missing Data:\n", missing_data)
print("\nDuplicates:", duplicates)

# Document missing data
missing_report = missing_data[missing_data > 0].to_dict()
print("\nMissing Data Report:", missing_report)

# Document duplicates
if duplicates > 0:
    data = data.drop_duplicates()
    print(f"{duplicates} duplicates removed. Remaining entries: {len(data)}")

# Profiling numerical columns
numerical_summary = data.describe()
print("\nNumerical Columns Summary:\n", numerical_summary)

# Check categorical columns for inconsistencies
categorical_columns = ['GHG', 'Sector', 'EDGAR Country Code', 'Country']
categorical_summary = {col: data[col].value_counts() for col in categorical_columns}
print("\nCategorical Columns Summary:\n", categorical_summary)

# Mixed-type columns and data types
data_types = data.dtypes
print("\nData Types:\n", data_types)

# Addressing missing data by averaging 5 years before and after for the country and sector
def fill_missing_data(row, year, data):
    sector = row['Sector']
    country = row['Country']
    start_year = max(1970, year - 5)
    end_year = min(2022, year + 5)
    
    relevant_years = list(range(start_year, end_year + 1))
    if year in relevant_years:
        relevant_years.remove(year)  # Remove the current year

    values = data[(data['Sector'] == sector) & 
                  (data['Country'] == country)][relevant_years].dropna()
    
    if not values.empty:
        return values.mean(axis=1).mean()  # Mean across the available years
    else:
        return np.nan

years = list(range(1970, 2023))
for year in years:
    col_name = str(year)
    if col_name in data.columns:
        data[col_name] = data.apply(
            lambda row: fill_missing_data(row, year, data) if pd.isnull(row[col_name]) else row[col_name], axis=1
        )

print("\nMissing data filled with the average of 5 years before and 5 years after for each country and sector.")

# Detecting and handling outliers (simple method using IQR)
def detect_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[col] < lower_bound) | (df[col] > upper_bound)]

outliers = {col: detect_outliers(data, col) for col in data.columns[4:]}

print("\nOutliers detected in each column:")
for col, outlier_df in outliers.items():
    print(f"{col}: {len(outlier_df)} outliers")

# Handling outliers (removing in this case)
for col in data.columns[4:]:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]

print("\nOutliers removed from the dataset.")
print("\nCleaned data shape:", data.shape)

# Save the cleaned data
output_file_path = '/Users/ging/Documents/Data Analytics Course/GHG Emissions Worldwide/02 Data/Prepared Data/GHG_Emissions_World_Sector_Country_Cleaned.xlsx'
data.to_excel(output_file_path, index=False)
print(f"\nCleaned data saved to {output_file_path}")

# Summary statistics as a simple table
summary_stats = data.describe().transpose()
print("\nSummary Statistics:")
print(summary_stats)

summary_stats_output_path = '/Users/ging/Documents/Data Analytics Course/GHG Emissions Worldwide/04 Analysis/Reports/Summary_Statistics.xlsx'
summary_stats.to_excel(summary_stats_output_path)
print(f"\nSummary statistics saved to {summary_stats_output_path}")



Missing Data:
 GHG                     0
Sector                  0
EDGAR Country Code      0
Country                 0
1970                  130
1971                  130
1972                  128
1973                  121
1974                  123
1975                  121
1976                  112
1977                  111
1978                  106
1979                  104
1980                  118
1981                  117
1982                  118
1983                  112
1984                  113
1985                  109
1986                  109
1987                  109
1988                  109
1989                  111
1990                  106
1991                  112
1992                  107
1993                  107
1994                  106
1995                  101
1996                   95
1997                   96
1998                   98
1999                  103
2000                  109
2001                   95
2002                  102
2003                   


Cleaned data saved to /Users/ging/Documents/Data Analytics Course/GHG Emissions Worldwide/02 Data/Prepared Data/GHG_Emissions_World_Sector_Country_Cleaned.xlsx

Summary Statistics:
      count      mean       std           min       25%       50%       75%  \
1970  447.0  0.000187  0.000645  0.000000e+00  0.000003  0.000027  0.000149   
1971  447.0  0.000188  0.000645  0.000000e+00  0.000003  0.000027  0.000149   
1972  447.0  0.000183  0.000626  0.000000e+00  0.000004  0.000028  0.000156   
1973  447.0  0.000194  0.000671  0.000000e+00  0.000004  0.000034  0.000162   
1974  447.0  0.000194  0.000640  0.000000e+00  0.000005  0.000036  0.000164   
1975  447.0  0.000183  0.000506  0.000000e+00  0.000005  0.000035  0.000171   
1976  447.0  0.000184  0.000526  0.000000e+00  0.000009  0.000042  0.000169   
1977  447.0  0.000193  0.000541  0.000000e+00  0.000010  0.000045  0.000175   
1978  447.0  0.000180  0.000410  0.000000e+00  0.000010  0.000044  0.000181   
1979  447.0  0.000188  0.000